In [11]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

def generator():
    while True:
        yield

In [15]:
index = 0

all_a = []
for _ in tqdm(generator()):
    r = requests.get(f'https://www.islam.gov.my/ms/e-penerbitan?start={index}')
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if '/ms/e-penerbitan/' in a_.get('href', '')]
    all_a.extend(a)
    index += 10

In [16]:
all_a = list(set(all_a))
all_a

['/ms/e-penerbitan/2217-kit-panduan-hari-raya-di-rumah',
 '/ms/e-penerbitan/2184-laporan-tahunan-jakim-2015',
 '/ms/e-penerbitan/2483-tadabbur-kalamullah-16-ramadhan-1441h-berbaktilah-kepada-ibubapa',
 '/ms/e-penerbitan/2513-tadabbur-kalamullah-17-syawal-1441h-kerugian-memilih-kekufuran-dan-kederhakaan',
 '/ms/e-penerbitan/2156-konsep-rahmah-dalam-sosioekonomi',
 '/ms/e-penerbitan/2160-konsep-rahmah-dalam-tadbir-urus',
 '/ms/e-penerbitan/3802-laporan-tahunan-jakim-2022',
 '/ms/e-penerbitan/2511-tadabbur-kalamullah-15-syawal-1441h-ingatlah-akan-kematian',
 '/ms/e-penerbitan/3946-daftar-buku-naskhah-hadiah-dan-percuma-terbitan-jakim-kemas-kini-oktober-2023',
 '/ms/e-penerbitan/2208-risalah-maulidur-rasul-1437h',
 '/ms/e-penerbitan/2516-tadabbur-kalamullah-20-syawal-1441h-ikutilah-petunjuk-al-quran',
 '/ms/e-penerbitan/3358-buletin-ikmas-jakim-bilangan-1-tahun-2022',
 '/ms/e-penerbitan/3337-keluarga-malaysia-keluarga-sejahtera',
 '/ms/e-penerbitan/2781-risalah-sambutan-ma-al-hijrah-pering

In [28]:
pdfs = []
for a in tqdm(all_a):
    url = f'https://www.islam.gov.my{a}'
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = ['https://www.islam.gov.my' + a_.get('href') for a_ in a if a_.get('href', '').endswith('.pdf')]
    pdfs.extend(a)

100%|█████████████████████████████████████████| 290/290 [02:21<00:00,  2.04it/s]


In [29]:
pdfs = list(set(pdfs))

In [30]:
len(pdfs)

180

In [32]:
import json

with open('pdfs.json', 'w') as fopen:
    json.dump(pdfs, fopen)

In [33]:
!mkdir e-penerbitan

In [39]:
import os

for i in tqdm(range(len(pdfs))):
    url = pdfs[i]
    filename = os.path.join('e-penerbitan', os.path.split(url)[-1])
    r = requests.get(url)
    with open(filename, 'wb') as fopen:
        fopen.write(r.content)

100%|█████████████████████████████████████████| 180/180 [03:43<00:00,  1.24s/it]


In [43]:
from tika import parser
from bs4 import BeautifulSoup

TIKA_HOST = 'http://localhost:9998'

def parse_tika(f, minlen=2,):
    while True:
        try:
            raw_xml = parser.from_file(f, TIKA_HOST, xmlContent=True)
            break
        except BaseException:
            time.sleep(1.0)
    body = BeautifulSoup(raw_xml['content']).find('body')
    return '\n'.join(body.findAll(text=True))

In [45]:
with open('e-penerbitan.jsonl', 'w') as fopen:
    for f in tqdm(glob('e-penerbitan/*.pdf')):
        t = parse_tika(f)
        d = {
            'text': t,
            'filename': f
        }
        fopen.write(f'{json.dumps(d)}\n')

100%|█████████████████████████████████████████| 180/180 [03:51<00:00,  1.29s/it]
